In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

2023-05-20 08:11:03.389110: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 08:11:03.722191: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-20 08:11:03.823539: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# check gpu

tf.test.gpu_device_name()

2023-05-20 08:11:07.060663: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 08:11:08.425566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-20 08:11:08.436187: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-20 08:11:08.436425: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been bu

'/device:GPU:0'

In [3]:
device = tf.test.gpu_device_name()
device

2023-05-20 08:11:09.404662: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-20 08:11:09.405029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-20 08:11:09.405296: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-20 08:11:09.405749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-20 08:11:09.405768: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1700] Could not ident

'/device:GPU:0'

In [4]:
# devices = tf.config.list_physical_devices('GPU')
# devices

In [5]:

path = r"/wickkiey_git/InputVideos/2.mp4"


In [6]:
import cv2

def read_random_frame(video_path, resize_factor=4):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Generate a random frame number
    random_frame_number = int(np.random.randint(0, total_frames))

    # Set the frame position to the random frame number
    video.set(cv2.CAP_PROP_POS_FRAMES, random_frame_number)

    # Read the frame
    ret, frame = video.read()

    # Check if the frame was successfully read
    if not ret:
        raise ValueError("Error reading video frame")

    # Resize the frame to 4 times smaller
    resized_frame = cv2.resize(frame, (frame.shape[1] // resize_factor, frame.shape[0] // resize_factor))

    # Release the video capture object
    video.release()
    
    # convert frames to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    
    # return tensorflow tensors with extended dimension
    resized_frame, frame =  tf.expand_dims(resized_frame, axis=0), tf.expand_dims(frame, axis=0)

    return tf.cast(resized_frame,tf.float32), tf.cast(frame, tf.float32)



In [7]:
resized_frame, original_frame = read_random_frame(path)



2023-05-20 08:11:09.753628: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-20 08:11:09.753988: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-20 08:11:09.754245: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-20 08:11:09.754859: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-20 08:11:09.755173: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not op

In [8]:
resized_frame.shape, original_frame.shape

(TensorShape([1, 180, 320, 3]), TensorShape([1, 720, 1280, 3]))

In [17]:

initializer = tf.random_normal_initializer(0., 0.02)

upscale = tf.keras.Sequential([
    tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', kernel_initializer=initializer, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', kernel_initializer=initializer, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (3, 3), padding='same', activation='sigmoid'),
])

In [19]:
upscale.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.05), loss='mse', metrics=['mse'])

In [20]:
# get number of learnable parameters
# upscale.summary()

In [21]:
# conver model to gp

In [22]:
# fit a tensorflow generator 

def generator():
    while True:
        yield read_random_frame(path)

In [23]:
with tf.device(device):
    upscale.fit_generator(generator(), steps_per_epoch=100, epochs=20)

/tmp/ipykernel_343/632153096.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  upscale.fit_generator(generator(), steps_per_epoch=100, epochs=20)


Epoch 1/20
100/100 [==============================] - 7s 67ms/step - loss: 18670.6680 - mse: 18670.6660
Epoch 2/20
100/100 [==============================] - 7s 68ms/step - loss: 17737.5273 - mse: 17737.5234
Epoch 3/20
100/100 [==============================] - 7s 67ms/step - loss: 19633.5312 - mse: 19633.5312
Epoch 4/20
100/100 [==============================] - 7s 65ms/step - loss: 17216.7324 - mse: 17216.7344
Epoch 5/20
100/100 [==============================] - 7s 66ms/step - loss: 17728.0742 - mse: 17728.0762
Epoch 6/20
100/100 [==============================] - 7s 70ms/step - loss: 17921.6797 - mse: 17921.6777
Epoch 7/20
100/100 [==============================] - 7s 72ms/step - loss: 18510.9336 - mse: 18510.9355
Epoch 8/20
100/100 [==============================] - 7s 66ms/step - loss: 17875.8730 - mse: 17875.8750
Epoch 9/20
100/100 [==============================] - 7s 67ms/step - loss: 17111.0645 - mse: 17111.0684
Epoch 10/20
100/100 [==============================] - 7s 72ms/s